# Understanding the NAVAR code

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/navar/NAVAR

Mounted at /content/drive
/content/drive/MyDrive/navar/NAVAR


Installing Bussman's requirements:

In [ ]:
!pip install -r requirements.txt

In [4]:
from train_NAVAR import train_NAVAR
from evaluate import calculate_AUROC, dream_file_to_causal_matrix
import pandas as pd
import argparse

To understand the code better, we will just run a single experiment, in particular the one called `ecoli1`.

In [5]:
experiment = 'ecoli1' # experiment name
# should accuracy be evaluated?
evaluate = True
# should we use LSTM?
lstm = True 
lambda1 = 0.26025947107502856
batch_size = 46 # batch size for training
wd = 1.4961159190152877e-05
hidden_nodes = 10 # random (think width of layer)
learning_rate = 0.002 
hl = 1 # number of hidden layers (LSTM: number of recurrent states)

In [6]:
# load the data
file = f'experiments/{experiment}.tsv'
ground_truth_file = f'experiments/{experiment}_gt.txt'
data = pd.read_csv(file, sep='\t')
data = data.values[:, 1:]
epochs = 100
maxlags = 21 if lstm else 2

Let's explore that data for a moment. The `ecoli1` data is made up by 966 rows and 100 columns.

In [7]:
print(data.shape)
data

(966, 100)


array([[0.15213198, 0.05465604, 0.04711906, ..., 0.13579787, 0.39367317,
        0.39847743],
       [0.2012258 , 0.08574432, 0.20401708, ..., 0.23993862, 0.48825811,
        0.35958879],
       [0.19322354, 0.02080837, 0.32134769, ..., 0.27688198, 0.5669294 ,
        0.37422256],
       ...,
       [0.13009743, 0.09215208, 0.45135629, ..., 0.42214107, 0.93147055,
        0.48743001],
       [0.21391873, 0.01804852, 0.48076942, ..., 0.40894357, 0.85753113,
        0.43878193],
       [0.17129875, 0.0600136 , 0.43334571, ..., 0.4263017 , 0.85685361,
        0.50165747]])

In [13]:
%%time
# start training
print(f"Starting training on the data from experiment {experiment}, training for {epochs} iterations.")
score_matrix, _, _ = train_NAVAR(data, maxlags=maxlags, hidden_nodes=hidden_nodes, dropout=0, epochs=epochs,
                                 learning_rate=learning_rate, batch_size=batch_size, lambda1=lambda1,
                                 val_proportion=0.0, weight_decay=wd, check_every=500, hidden_layers=hl, normalize=True,
                                 split_timeseries=21, lstm=lstm)
# evaluate
print('Done training!')
if evaluate:
    ground_truth_matrix = dream_file_to_causal_matrix(ground_truth_file)
    AUROC = calculate_AUROC(score_matrix, ground_truth_matrix, ignore_self_links=True)
    print(f"The AUROC of this model on experiment {experiment} is: {AUROC}")

Starting training on the data from experiment ecoli1, training for 100 iterations.
Done training!
The AUROC of this model on experiment ecoli1 is: 0.5700026598465473
CPU times: user 52.6 s, sys: 23.8 s, total: 1min 16s
Wall time: 1min 23s


In [8]:
# Assgin parameters:
dropout=0
check_every=500
split_timeseries=21
val_proportion=0.1
normalize=True
hidden_layers=hl
weight_decay=wd

In [9]:
import torch
from NAVAR import NAVARLSTM
# T is the number of time steps, N the number of variables
T, N = data.shape
# instantiate the NAVAR model
model = NAVARLSTM(N, hidden_nodes, maxlags, dropout=dropout, hidden_layers=hidden_layers)
# use Mean Squared Error and the Adam optimzer
criterion = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

## Data loading

### Prepare data

In [10]:
data_raw = data
data = torch.from_numpy(data)

# normalize every variable to have 0 mean and standard deviation 1 -- necessary?
if normalize:
    data = data / torch.std(data, dim=0)
    data = data - data.mean(dim=0)

In [11]:
# initialize our input and target variables
X = torch.zeros((int(T/maxlags), maxlags, N)) # input 
X.shape # what does the first dim mean?

torch.Size([46, 21, 100])

In [12]:
# X and Y consist of timeseries of length K
for i in range(int(T/maxlags) -1):
    X[i, :, :] = data[i*maxlags:(i+1)*maxlags, :] # batches?

In [13]:
X = X.permute(0, 2, 1) # just transposes matrices
X.shape

torch.Size([46, 100, 21])

In [14]:
X.view(-1, N, maxlags)
Y = X[:, :, 1:]
X = X[:, :, :-1]

### Train test split

In [15]:
import numpy as np
number_of_val_indices = np.int(np.floor(val_proportion * Y.shape[0]))
number_of_val_indices

4

In [16]:
train_indices = np.arange(Y.shape[0] - number_of_val_indices)
val_indices = np.arange(Y.shape[0] - number_of_val_indices, Y.shape[0])

In [17]:
if val_proportion == 0:
    X_train = X
    Y_train = Y
else:
    X_train = X[train_indices]
    Y_train = Y[train_indices]
    X_val = X[val_indices]
    Y_val = Y[val_indices]

## Back to training

Check of cuda GPU available and move to it. Available on Colab yay

In [18]:
# push model and data to GPU if available
if torch.cuda.is_available():
    model = model.cuda()
    X_train = X_train.cuda()
    Y_train = Y_train.cuda()
    if X_val is not None:
        X_val = X_val.cuda()
        Y_val = Y_val.cuda()

In [19]:
# Initialize
num_training_samples = X_train.shape[0]
total_loss = 0
loss_val = 0

This is where the actual training happens now.

In [ ]:
# start of training loop
batch_counter = 0
for t in range(1, epochs+1): # loop over epochs / iterations
    #obtain batches
    batch_indeces_list = []
    # if desired number of batches for training is smaller than 
    # the number of training samples, then enter here:
    if batch_size < num_training_samples: 
        batch_perm = np.random.choice(num_training_samples, size=num_training_samples, replace=False)
        for i in range(int(num_training_samples/batch_size) + 1):
            start = i*batch_size
            batch_i = batch_perm[start:start+batch_size]
            if len(batch_i) > 0:
                batch_indeces_list.append(batch_perm[start:start+batch_size])
    else:
        batch_indeces_list = [np.arange(num_training_samples)]

    # Now train for each training batch:
    for batch_indeces in batch_indeces_list:
        batch_counter += 1 # increase counter
        X_batch = X_train[batch_indeces] # get training data
        Y_batch = Y_train[batch_indeces] # ...
        
        # forward pass to calculate predictions and contributions
        predictions, contributions = model(X_batch) # return predictions and contributions

        # calculate the loss
        loss_pred = criterion(predictions, Y_batch) # basically loss on 1-step ahead prediction
        loss_l1 = (lambda1/N) * torch.mean(torch.sum(torch.abs(contributions), dim=1))
        loss = loss_pred + loss_l1
        total_loss += loss

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # every 'check_every' epochs we calculate and print the validation loss
    if t % check_every == 0:
        model.eval()
        if val_proportion > 0.0:
            val_pred, val_contributions = model(X_val)
            loss_val = criterion(val_pred, Y_val)
        model.train()

        print(f'iteration {t}. Loss: {total_loss/batch_counter}  Val loss: {loss_val}')
        total_loss = 0
        batch_counter = 0

For a single epoch ...

In [43]:
# start of training loop
batch_counter = 0
t = 1
#obtain batches
batch_indeces_list = []
# if desired number of batches for training is smaller than 
# the number of training samples, then enter here:
if batch_size < num_training_samples: 
    batch_perm = np.random.choice(num_training_samples, size=num_training_samples, replace=False)
    for i in range(int(num_training_samples/batch_size) + 1):
        start = i*batch_size
        batch_i = batch_perm[start:start+batch_size]
        if len(batch_i) > 0:
            batch_indeces_list.append(batch_perm[start:start+batch_size])
else:
    batch_indeces_list = [np.arange(num_training_samples)]

... for a single batch:

> Here we noted a weird thing: the batch size here really refers to how many blocks of size ($N \times$ `maxlags`) we want to include in each batch. So the number of total observations per batch is really the specified `batch_size` multiplied by the specified maximum number of lags `maxlags`. We noted that depending on specifications only a single or very few train-test splits will be performed in each epoch. Does that cause an inefficiency?

For example, for his choices, the first and only batch here simply corresponds to all of the training data.

In [45]:
batch_counter += 1 # increase counter
X_batch = X_train[batch_indeces] # get training data
Y_batch = Y_train[batch_indeces] # ...

Now we will go into the model:

In [46]:
# forward pass to calculate predictions and contributions
predictions, contributions = model(X_batch) # return predictions and contributions

# calculate the loss
loss_pred = criterion(predictions, Y_batch) # basically loss on 1-step ahead prediction
loss_l1 = (lambda1/N) * torch.mean(torch.sum(torch.abs(contributions), dim=1))
loss = loss_pred + loss_l1
total_loss += loss

# Zero gradients, perform a backward pass, and update the weights.
optimizer.zero_grad()
loss.backward()
optimizer.step()